In [1]:
print('this is test file for a new repository')

this is test file for a new repository


In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

conn = create_engine('')


# определяем функцию получения данных из первичного источника
# она получает на вход объект соединения к первичному источнику
# и возвращает данные из всех таблиц, собранные в одном pandas.DataFrame
def extract(cnx) -> pd.DataFrame:
  # сначала напишите SQL-запрос, который объединяет все таблицы в одну
	sql_query = '''SELECT contracts.customer_id, begin_date, end_date, type,
    paperless_billing, payment_method, monthly_charges,
    total_charges,internet_service,online_security,
    online_backup,device_protection,tech_support,streaming_tv,
    streaming_movies,gender,senior_citizen,partner,
    dependents,multiple_lines FROM contracts
    Left join internet on internet.customer_id = contracts.customer_id 
    Left join personal on personal.customer_id = contracts.customer_id
    Left join phone on phone.customer_id = contracts.customer_id'''
	data = pd.read_sql(sql_query, cnx) #исполним написанный запрос
	return data
    
	

In [ ]:
def transform(data: pd.DataFrame) -> pd.DataFrame:
    data['end_date'] = data['end_date'].replace('No', None)
    data['target'] = pd.notnull(data['end_date'])
    data['target'] = data['target'].astype(int)
    
    return data

In [ ]:
conn = create_engine('')

def load(data: pd.DataFrame, db_conn: sqlalchemy.engine.base.Engine, table_name: str = 'users_churn'):
    data.to_sql(name=table_name, con=db_conn)


In [ ]:
import pandas as pd
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.decorators import task
# ваш код здесь
@task()
def load(data: pd.DataFrame):
    hook = PostgresHook('destination_db')
    conn = hook.get_conn()
    hook.insert_rows(
            table="users_churn",
            replace=True,
            target_fields=data.columns.tolist(),
            replace_index=['customer_id'],
            rows=data.values.tolist()
    )
    conn.close()

In [ ]:
import pandas as pd
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.decorators import task, dag
from pendulum import datetime
# ваш код здесь
@dag(schedule='@once',start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),tags=["churn"]) # обозначаем, что это DAG, и определяем параметры
def prepare_churn_dataset():
    # ваш код здесь
    @task() #extract
    def extract() -> pd.DataFrame:
        hook = PostgresHook('source_db')
        conn = hook.get_conn()
        sql_query = f"""
            select
                c.customer_id, c.begin_date, c.end_date, c.type, c.paperless_billing, c.payment_method, c.monthly_charges, c.total_charges,
                i.internet_service, i.online_security, i.online_backup, i.device_protection, i.tech_support, i.streaming_tv, i.streaming_movies,
                p.gender, p.senior_citizen, p.partner, p.dependents,
                ph.multiple_lines
            from contracts as c
            left join internet as i on i.customer_id = c.customer_id
            left join personal as p on p.customer_id = c.customer_id
            left join phone as ph on ph.customer_id = c.customer_id
            """
        data = pd.read_sql(sql_query, conn) #исполним написанный запрос
        hook.close()
        return data

    @task() #transform
    def transform(data: pd.DataFrame) -> pd.DataFrame:
        data['target'] = (data['end_date'] != 'No').astype(int)
        data['end_date'].replace({'No': None}, inplace=True)
        return data
    
    @task() #load
    def load(data: pd.DataFrame):
        hook = PostgresHook('destination_db')
        conn = hook.get_conn()
        hook.insert_rows(
            table="users_churn",
            replace=True,
            target_fields=data.columns.tolist(),
            replace_index=['customer_id'],
            rows=data.values.tolist()
        )
        conn.close()
    data = extract()
    transformed_data = transform(data)
    load(transformed_data)